# Not Quite Done Yet

#### TO DO:
* Figure out some way to get a randomized buffer for the training/validation set

### Stuff for Notebook Start

In [ ]:
!pip install pytorch_lightning
!pip install torchvision

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('drive/MyDrive/colab_insanity')

### Actual Code starts HerE

In [23]:
import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
import copy
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torchvision as tv
import torch
from torch.utils.data import Dataset, DataLoader

In [24]:
# read in data from the .csv

data = pd.read_csv("consolidated_tuna_data_clean.csv",thousands=',',decimal='.')

# eliminate the period, put in underscore
data.columns = data.columns.str.replace('.','')

In [64]:
class TunaTimeseries(Dataset):

    def __init__(self, X: pd.DataFrame, y: pd.DataFrame, sequence: int = 1, train=False):
        self.X = torch.tensor(X.to_numpy()).float()
        self.y = torch.tensor(y.to_numpy()).float()
        self.sequence = sequence

    def __len__(self):
        return self.X.__len__() - (self.sequence-1)

    def __getitem__(self, index):
        return (self.X[index:index+self.sequence], self.y[index:index+self.sequence])

In [70]:
class TunaFishModule(pl.LightningDataModule):

    def __init__(self, data_dir: str= './', batch_size=4,
                 sequence=12, hold_test=1, hold_valid=1):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # time series params
        self.sequence=sequence
        self.hold_test=hold_test
        self.hold_valid=hold_valid

    def prepare_data(self, data_name = 'consolidated_tuna_data_clean.csv'):
        data = pd.read_csv(data_name,thousands=',',decimal='.')
        data["time_idx"] = data["year"]* 12 + data["month"]
        data["time_idx"] -= data["time_idx"].min()
        data["month"] = data["month"].astype(float)
        data["year"] = data["year"].astype(float)
        data["skipjack_tuna_price"] = data["skipjack_tuna_price"].astype(float)
        # lag the target variable
        data['skipjack_tuna_price'] = data['skipjack_tuna_price'].shift(-1)

        self.y = data['skipjack_tuna_price']
        self.X = data.drop(columns=['skipjack_tuna_price'])
    
    def setup(self, stage = 'None'):
        y = self.y
        X = self.X

        # drop the last row because we're trying for prediction
        y = y.drop([len(y)-1])
        X = X.drop([len(X)-1])

        # pull out the test set based on sequence length and desired hodl
        idx_test_end = len(y)-1
        idx_test_srt = len(y)-1-(self.hold_test*self.sequence)

        y_test = y.iloc[idx_test_srt:idx_test_end]
        X_test = X.iloc[idx_test_srt:idx_test_end]
        y_in = y.iloc[:idx_test_end]
        X_in = X.iloc[:idx_test_end]

        # pull out the validation set based on sequence length and desired hodl
        idx_val_end = len(y)-1
        idx_val_srt = len(y)-1-(self.hold_valid*self.sequence)

        y_val   = y_in.iloc[idx_val_srt:idx_val_end]
        X_val   = X_in.iloc[idx_val_srt:idx_val_end]
        y_train = y_in.iloc[:idx_val_end]
        X_train = X_in.iloc[:idx_val_end]

        # training / val datasets
        if stage == 'fit' or stage is None:
            self.tuna_train = TunaTimeseries(X_train, y_train, self.sequence, train=True)
            self.tuna_val   = TunaTimeseries(X_val, y_val, self.sequence, train=True)

        if stage == 'test' or stage is None:
            self.tuna_test = TunaTimeseries(X_test, y_test, self.sequence, train=False)
            

    def train_dataloader(self):
        return DataLoader(self.tuna_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.tuna_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.tuna_test, batch_size=self.batch_size)

In [71]:
tuna = TunaFishModule()
tuna.prepare_data()
tuna.setup(stage='fit')

In [72]:
for i,(X,y) in enumerate(tuna.train_dataloader()):
    if i==1:
        print(X.shape)
        print(y.shape)
        print()
        print(X[:,:,432]) # time index sequence
        print(y[:,:]) # target sequence (skipjack price)

torch.Size([4, 12, 433])
torch.Size([4, 12])

tensor([[10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21.],
        [72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82., 83.],
        [85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95., 96.],
        [49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60.]])
tensor([[2005., 2030., 1930., 2090., 2195., 2198., 2200., 2240., 2280., 2350.,
         2210., 1900.],
        [1700., 1500., 1690., 1700., 1900., 1950., 1980., 2100., 2300., 2000.,
         1800., 1550.],
        [1700., 1800., 1600., 1600., 1300., 1450., 1650., 1525., 1400., 1300.,
         1280., 1480.],
        [1010., 1035., 1070., 1200., 1400., 1500., 1390., 1140., 1000., 1000.,
         1000., 1170.]])
